# Naive Bayes Classifier — Versione Didattica

Questo notebook contiene una versione **semplice e didattica** del classificatore Naive Bayes,
insieme a un piccolo **dataset di fiori** per testarlo.


## 📘 1. Definizione del modello Naive Bayes.

In [14]:
class SimpleNaiveBayes:
    """
    CLASSIFICATORE NAIVE BAYES (versione didattica)

    Questo modello impara a classificare esempi in base alle loro caratteristiche
    usando le probabilità. Non fa supposizioni complicate: considera
    "naivamente" che ogni caratteristica sia indipendente dalle altre.

    L'idea è:
        P(Classe | Caratt_i)  ∝  P(Classe) × P(Caratteristica_1 | Classe) × ...

    Questa classe implementa esattamente questo concetto con passaggi molto semplici.
    """

    def __init__(self):

        """
        Inizializzazione dell'oggetto.
        
        Creiamo due strutture dati fondamentali:
        - self.prior → contiene P(Classe)
        - self.feature_probs → contiene P(feature | classe)
        
        Esempio struttura finale:
        prior = {"rosa": 0.33, "girasole": 0.33, "margherita": 0.33} quindi quanto spesso compare ogni classe, ad es per i fiori, quante rose?

        feature_probs = {
            "rosa": [
                {"rosso": 1.0, "giallo": 0.0},           # Probabilità del colore dato "rosa"
                {"tonda": 1.0, "allungata": 0.0}         # Probabilità della forma dato "rosa"
            ],
            "margherita": [...],
            "girasole": [...]
            contiene quanto spesso una certa caratteristica appare
           dentro una classe.
           Tipo: di tutte le rose, quante sono rosse?
        }
        """

        self.prior = {}             # Probabilità delle classi (P(classe))
        self.feature_probs = {}     # Probabilità delle feature (P(feature | classe))

# -------------------------------------------------------------------------
#                               FIT (ADDESTRAMENTO)
# -------------------------------------------------------------------------


    def fit(self, X, y):

        """
        Addestra il modello sui dati.

        X: lista di tuple che rappresentano le caratteristiche
           Esempio: [("rosso", "tonda"), ("giallo", "allungata"), ...]

        y: lista delle classi corrispondenti
           Esempio: ["rosa", "girasole", "margherita", ...]

        L'obiettivo dell'addestramento è:
            1) Contare quante volte compare ogni classe → P(Classe)
            2) Per ogni classe, contare come sono distribuiti i valori
               delle feature → P(Feature | Classe)
        """

        n = len(y)                  # Numero totale di esempi
        classi = set(y)             # Le classi distinte (es: {"rosa", "margherita", "girasole"}) senza duplicati
        n_features = len(X[0])      # Numero di caratteristiche (feature)


        # -------------------------------------------------------------------------
        # 1. CALCOLO DELLE PROBABILITÀ A PRIORI: P(Classe)
        # -------------------------------------------------------------------------
        # P(classe)
        for cls in classi:                      # Quanti esempi appartengono a questa classe? es quante volte compare "rosa" nella lista y?
            self.prior[cls] = y.count(cls) / n  # P(classe) = frequenza relativa - es quante volte compare "rosa" nella lista y?

        # -------------------------------------------------------------------------
        # 2. CALCOLO DELLE PROBABILITÀ CONDIZIONATE: P(feature | classe)
        # -------------------------------------------------------------------------
        # P(feature | classe)
        for cls in classi:

            # Prepara una lista vuota: un dizionario per ogni feature
            # Esempio: per 2 feature → [ {}, {} ]

            self.feature_probs[cls] = [ {} for _ in range(n_features) ]

            # Seleziona SOLO gli esempi della classe corrente
            # Esempio: per "rosa" prendo solo gli X[i] dove y[i] == "rosa"

            X_cls = [X[i] for i in range(n) if y[i] == cls]
            n_cls = len(X_cls)

            # Per ogni caratteristica

            for f_idx in range(n_features):

                # Prendo i valori della feature f_idx per questa classe
                # Esempio: per feature "colore" → ["rosso", "rosso"] se entrambe le rose sono rosse

                valori = [sample[f_idx] for sample in X_cls]

                # Considero ogni valore possibile
                for v in set(valori):
                    # P(v | cls) = numero di volte che v compare / numero esempi classe
                    self.feature_probs[cls][f_idx][v] = valori.count(v) / n_cls

    # -------------------------------------------------------------------------
    #                               PREVISIONE
    # -------------------------------------------------------------------------

    def predict_proba(self, x):
        """
        Qui il modello cerca di capire A QUALE CLASSE APPARTIENE
        un nuovo esempio x (es. un fiore).
        Calcola P(Classe | x) per OGNI classe.

        x: nuovo esempio da classificare (es: ("giallo", "tonda"))

        Procedura:
            Per ogni classe:
                1. Parto da P(Classe)
                2. Moltiplico per ogni P(feature | classe)
                3. Se un valore non è stato mai visto → smoothing (0.001)
            Alla fine normalizzo tutto per ottenere probabilità vere che sommano a 1.
        """
        post = {}                   # Dizionario che conterrà le probabilità non normalizzate
        for cls in self.prior:
            # 1) Partenza: P(Classe)
            prob = self.prior[cls]

            # 2) Moltiplicazione delle probabilità condizionate
            for f_idx, valore in enumerate(x):
                # Se abbiamo già visto il valore durante l'addestramento:
                if valore in self.feature_probs[cls][f_idx]:
                    prob *= self.feature_probs[cls][f_idx][valore]
                else:
                    # Valore mai visto → Laplace smoothing molto semplice
                    # Serve a evitare che una sola probabilità zero annulli tutto.
                    prob *= 0.001
            post[cls] = prob

        # -------------------------------------------------------------------------
        #  NORMALIZZAZIONE
        #  Convertiamo le probabilità “grezze” in probabilità vere che sommano a 1.
        # -----------------------------------------------------------------------
        tot = sum(post.values())
        return {cls: post[cls] / tot for cls in post}


    # -------------------------------------------------------------------------
    #                 RESTITUISCE SOLO LA CLASSE PIÙ PROBABILE
    # -------------------------------------------------------------------------
    def predict(self, x):
        """
        Restituisce la classe con probabilità più alta.
        """
        probs = self.predict_proba(x)
        return max(probs, key=probs.get)

## 2. Dataset dei fiori
Esempio semplice con 3 classi: rosa, margherita, girasole.

In [15]:
# Dataset didattico dei fiori
X = [
    ("rosso",  "tonda"),
    ("bianco", "tonda"),
    ("giallo", "allungata"),
    ("giallo", "tonda"),
    ("rosso",  "tonda"),
    ("giallo", "allungata")
]

y = ["rosa", "margherita", "girasole", "margherita", "rosa", "girasole"]

X, y

([('rosso', 'tonda'),
  ('bianco', 'tonda'),
  ('giallo', 'allungata'),
  ('giallo', 'tonda'),
  ('rosso', 'tonda'),
  ('giallo', 'allungata')],
 ['rosa', 'margherita', 'girasole', 'margherita', 'rosa', 'girasole'])

##  3. Addestriamo il modello

In [16]:
clf = SimpleNaiveBayes()
clf.fit(X, y)

print("Modello addestrato!")

Modello addestrato!


##  4. Facciamo una previsione
Fiore nuovo: **giallo, tonda**

In [17]:
nuovo = ("giallo", "tonda")

print("Probabilità:", clf.predict_proba(nuovo))
print("Classe predetta:", clf.predict(nuovo))

Probabilità: {'rosa': 0.00199203187250996, 'margherita': 0.99601593625498, 'girasole': 0.00199203187250996}
Classe predetta: margherita
